In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.storage import LocalFileStore
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
)

memory = ConversationBufferMemory(
    return_messages=True,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader(file_path="./files/challenges/document.txt")
doc = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cache_embedding = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vector_store = FAISS.from_documents(doc, cache_embedding)

retriver = vector_store.as_retriever()


def load_memory(_):
    return memory.load_memory_variables({})["history"]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
    You are good at summarizing and explaining things you have read. expectially novels and storises. Can you summarize the document i gave to you for me? If you can't do when processing document that I gave to you, please let me know.
    -------
    {document}
    """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


chain = (
    {
        "document": retriver,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print("Question: ", question)
    print("Answer:", result.content)
    print("\n")


invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write on the table?")
invoke_chain("Who is Julla?")

Question:  Is Aaronson guilty?
Answer: In the context of George Orwell's "1984," Aaronson, along with Jones and Rutherford, is portrayed as having been falsely accused and convicted of crimes against the Party. Winston Smith, the protagonist, recalls a moment when he saw a photograph that proved their innocence, but he later convinces himself that the photograph never existed and that they were indeed guilty as charged. This reflects the Party's manipulation of truth and reality, as well as the concept of "doublethink," where individuals are forced to accept contradictory beliefs. Ultimately, Aaronson is not guilty of the crimes he was accused of, but the Party's control over information and memory leads to the acceptance of his guilt.


Question:  What message did he write on the table?
Answer: In "1984," Winston Smith traces the equation "2 + 2 = 5" in the dust on the table. This phrase symbolizes the Party's manipulation of truth and reality, illustrating how the regime can dictate 